In [1]:
!pip install -q faiss-cpu transformers tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 78.3 MB/s eta 0:00:00:00:0100:01


In [2]:
import json
import torch
import faiss
import numpy as np
from tqdm.auto import tqdm

from transformers import (
    DPRQuestionEncoder,
    DPRContextEncoder,
    DPRQuestionEncoderTokenizerFast,
    DPRContextEncoderTokenizerFast
)


2026-01-04 12:57:08.204006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767531428.369503     102 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767531428.416126     102 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767531428.806898     102 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767531428.806942     102 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767531428.806945     102 computation_placer.cc:177] computation placer alr

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [4]:
q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained(
    "facebook/dpr-question_encoder-single-nq-base"
)
p_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base"
)

q_encoder = DPRQuestionEncoder.from_pretrained(
    "facebook/dpr-question_encoder-single-nq-base"
).to(device)

p_encoder = DPRContextEncoder.from_pretrained(
    "facebook/dpr-ctx_encoder-single-nq-base"
).to(device)

q_encoder.eval()
p_encoder.eval()


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DPRContextEncoder(
  (ctx_encoder): DPREncoder(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768,

In [5]:
if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs")

    q_encoder = torch.nn.DataParallel(q_encoder)
    p_encoder = torch.nn.DataParallel(p_encoder)


Using 2 GPUs


In [6]:
import json
import random

def build_dpr_passages_capped(
    file_path,
    max_distractors_per_doc=5,
    seed=42
):
    random.seed(seed)
    passages = []

    with open(file_path, "r") as f:
        for line in f:
            ex = json.loads(line)

            doc_tokens = ex["document_text"].split()
            candidates = ex["long_answer_candidates"]
            ann = ex["annotations"][0]
            la = ann["long_answer"]

            # --- Gold passage ---
            gold_passage = None
            gold_idx = la.get("candidate_index", -1)

            if la["start_token"] != -1:
                gold_passage = " ".join(
                    doc_tokens[la["start_token"]:la["end_token"]]
                )
                passages.append(gold_passage)

            # --- Collect top-level distractors ---
            distractors = []
            for i, c in enumerate(candidates):
                if not c["top_level"]:
                    continue
                if i == gold_idx:
                    continue

                text = " ".join(
                    doc_tokens[c["start_token"]:c["end_token"]]
                )
                distractors.append(text)

            # --- Sample fixed number of distractors ---
            if distractors:
                sampled = random.sample(
                    distractors,
                    min(max_distractors_per_doc, len(distractors))
                )
                passages.extend(sampled)

    return passages


In [7]:
file_path = "/kaggle/input/the-natural-questions-dataset/simplified-nq-train.jsonl"

passages = build_dpr_passages_capped(
    file_path,
    max_distractors_per_doc=4
)

print("Total passages:", len(passages))
print("Example passage:\n", passages[0][:300])


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Total passages: 1368573
Example passage:
 <P> A common example of permission marketing is a newsletter sent to an advertising firm 's customers . Such newsletters inform customers of upcoming events or promotions , or new products . In this type of advertising , a company that wants to send a newsletter to their customers may ask them at th


In [8]:
def encode_passages(passages, batch_size=32, max_length=512):
    all_embeddings = []

    for i in tqdm(
        range(0, len(passages), batch_size),
        desc="Encoding passages (2 GPUs)"
    ):
        batch = passages[i:i + batch_size]

        inputs = p_tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=max_length,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = p_encoder(**inputs)
            # ✅ DPR embedding lives here
            emb = outputs.pooler_output

        emb = emb.cpu().numpy()
        faiss.normalize_L2(emb)   # ✅ cosine similarity

        all_embeddings.append(emb)

    return np.vstack(all_embeddings)


In [9]:
passage_embeddings = encode_passages(passages)
print("Embedding shape:", passage_embeddings.shape)

Encoding passages (2 GPUs):   0%|          | 0/42768 [00:00<?, ?it/s]

Embedding shape: (1368573, 768)


In [10]:
dim = passage_embeddings.shape[1]
index = faiss.IndexFlatIP(dim)

index.add(passage_embeddings)
faiss.write_index(index, "/kaggle/working/passage.index")

import pickle
pickle.dump(passages, open("/kaggle/working/passages.pkl", "wb"))

print("FAISS index size:", index.ntotal)

/kaggle/working/passage.index

FAISS index size: 1368573


In [11]:
def encode_question(question):
    inputs = q_tokenizer(
        question,
        return_tensors="pt",
        truncation=True
    ).to(device)

    with torch.no_grad():
        emb = q_encoder(**inputs).pooler_output  # ✅ correct for DPR

    emb = emb.cpu().numpy()
    faiss.normalize_L2(emb)
    return emb


In [12]:
def dpr_retrieve(question, k=5):
    q_emb = encode_question(question)
    scores, indices = index.search(q_emb, k)
    return [passages[i] for i in indices[0]]


In [13]:
question = "who is the mother in how i met your mother"

retrieved = dpr_retrieve(question, k=5)

for i, p in enumerate(retrieved, 1):
    print(f"\n--- Passage {i} ---")
    print(p[:400])


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



--- Passage 1 ---
<P> Tracy McConnell ( colloquial : `` The Mother '' ) is the title character from the CBS television sitcom How I Met Your Mother . The show , narrated by Future Ted , tells the story of how Ted Mosby met The Mother . Tracy McConnell appears in 8 episodes from `` Lucky Penny '' to `` The Time Travelers '' as an unseen character ; she was first seen fully in `` Something New '' and was promoted to 

--- Passage 2 ---
<P> Tracy McConnell ( colloquial : `` The Mother '' ) is the title character from the CBS television sitcom How I Met Your Mother . The show , narrated by Future Ted , tells the story of how Ted Mosby met The Mother . Tracy McConnell appears in 8 episodes from `` Lucky Penny '' to `` The Time Travelers '' as an unseen character ; she was first seen fully in `` Something New '' and was promoted to 

--- Passage 3 ---
<P> Tracy McConnell ( colloquial : `` The Mother '' ) is the title character from the CBS television sitcom How I Met Your Mother . The show 

In [14]:
scores, indices = index.search(encode_question("test question"), 3)
print(indices)


[[ 631127  956634  857923  594217 1146010]]


In [15]:
# embeddings are not collapsed
np.allclose(passage_embeddings[0], passage_embeddings[1])
# → False


False

In [16]:
index.ntotal == len(passages)
# → True


True

In [17]:
from IPython.display import FileLink

# This points to the file in your working directory
FileLink(r'passage.index')

/kaggle/working/passage.index

In [19]:
import pickle
pickle.dump(passages, open("/kaggle/working/passages.pkl", "wb"))
FileLink(r'passages.pkl')


/kaggle/working/passages.pkl